# Einbettung der Einwohnerzahlen in die Ortsinformationen

In [ ]:
import pandas as pd
from pathlib import Path

In [ ]:
input_data_folder = "../data/"
input_data_filename = "AAB_cleaned.csv"
output_data_filename = "Kfz_pro_Ort.csv"

data_file = Path(input_data_folder) / input_data_filename
output_file = Path(input_data_folder) / output_data_filename

In [ ]:
df_cars = pd.read_csv(data_file, sep= "\t")  

## Vorverarbeitung der Kfz-Daten 

> Achtung: Dieses Notebook ignoriert aus didaktischen Gründen viele Besonderheiten im Datensatz (mehrdeutige Ortsnamen, Schreibweisen) und vereinfacht den Analyseprozess. Die abgeleiteten Aussagen sind entsprechend nicht belastbar.

In [ ]:
df_counts = df_cars.groupby('Wohnort').agg(
             Anzahl=('Wohnort', 'count'),
             Kreis=('Kreis_Stadt', 'last'),
).sort_values(by='Anzahl', ascending=False).reset_index()
df_counts

In [ ]:
# Wie groß war die Anzahl der Fahrzeugeigentümer in Freiberg gleich?



In [ ]:
#  Lassen sie uns die kleinen Orte entfernen, die weniger als 50 Fahrzeugeigentümer haben.



## Erfassen der Gemeindedaten / Einwohnerzahlen

> Vielen Dank an Herrn Ulli Schubert (info@ulischubert.de) für die Bereitstellung der Daten!

In [ ]:
inhabitants_statistics_filename = "Gemeindeverzeichnis1910 mit EW.xlsx"
data_file = Path(input_data_folder) / inhabitants_statistics_filename
df_inh = pd.read_excel(data_file)

In [ ]:
# Wie viele Einwohner hat Freiberg 1910? Stimmt der Wert mit anderen Quellen überein?

> Soweit so gut, welche Probleme gibt es?

In [ ]:
df_inh[df_inh.Gemeindename=="München"]

In [ ]:
# Die nachfolgende Lösung ist wenig belastbar und generiert falsche Ergebnisse - eine Lösung anhand der Kreisnamen und Bezirke/Länder wäre wesentlich zielführender.
# lösche die Gemeinden, deren Namen mehrfach in Gemeindename vorkommt, behalte den größeren Einwohnerwert

df_inh = df_inh.sort_values(by='Einwohner 1910', ascending=False)
df_inh = df_inh.drop_duplicates(subset='Gemeindename', keep='first')

df_inh[df_inh.Gemeindename=="München"]

## Merge der Daten

In [ ]:
df = pd.merge(df_counts, df_inh[['Gemeindename', 'Einwohner 1910']], left_on='Wohnort', right_on='Gemeindename', how='left')

In [ ]:
# lösche alle Orte, die nicht in der Einwohnerstatistik vorhanden sind

df = df.dropna(subset=['Einwohner 1910'])
df

In [ ]:
df.to_csv(output_file, sep="\t", index=False)